In [639]:
%pip install dash-bootstrap-components
%pip install dash pandas plotly

import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
import dash_table
from dash import callback_context

!pip3 install psycopg2
import pandas as pd
import openpyxl
import re
import json
import psycopg2
from psycopg2 import sql
from datetime import datetime, timedelta


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [640]:
# # Load the data from the Excel file
# data_path = 'analyze.xlsx'
# df = pd.read_excel(data_path, sheet_name = 'Conversation Logs')
# session_df = pd.read_excel(data_path, sheet_name = 'Sessions')

In [641]:
def connect_db():
    return psycopg2.connect(
        dbname="pvpower",
        user="phuongpd",
        password="vdkvn22.05",
        host="157.66.46.53",
        port="5432"
    )
 
cur = connect_db().cursor()
cur.execute("SELECT * FROM users")
print(cur.fetchall())
cur.close()

def fetch_data_from_table(conn, table_name):
    try:
        # Tạo con trỏ để thực hiện truy vấn
        cur = conn.cursor()
        
        # Thực hiện truy vấn
        query = sql.SQL("SELECT * FROM {table};").format(table=sql.Identifier(table_name))
        cur.execute(query)
        
        # Lấy kết quả của truy vấn
        rows = cur.fetchall()
        
        # Lấy tên cột
        colnames = [desc[0] for desc in cur.description]
        
        # Đóng con trỏ
        cur.close()
        
        # Đưa dữ liệu vào DataFrame
        df = pd.DataFrame(rows, columns=colnames)
        
        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Đã xảy ra lỗi: {error}")
        return None

def update_table_in_db(df, table_name):
    """
    Updates a PostgreSQL table with data from a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data to be inserted.
    """
    # Establish the connection
    conn = connect_db()
    cur = conn.cursor()
    
    # Ensure the table exists or create it
    columns = ', '.join([f"{col} VARCHAR" for col in df.columns])  # Assumes all columns are VARCHAR, adjust as needed
    create_table_query = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        {columns}
    );
    '''
    cur.execute(create_table_query)
    conn.commit()
    
    # Insert DataFrame into the table
    for i, row in df.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        columns = ', '.join(row.index)
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cur.execute(insert_query, tuple(row))
    
    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()
    
    print(f"DataFrame inserted into {table_name} table in database")

def delete_table_in_db(deleted_table_name):
    """
    Delete a PostgreSQL table from a DataFrame.

    Parameters:
    deleted_table_name (pandas.DataFrame): The name of table to be deleted.
    """
    # Establish the connection
    conn = connect_db()
    cur = conn.cursor()
    
    # SQL command to delete the table
    delete_table_query = f"DROP TABLE IF EXISTS feedback_hanh;"

    # Execute the delete table query
    cur.execute(delete_table_query)
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()
    
    print(f"The {deleted_table_name} table has been deleted from database")

def create_bar_chart(dataframe, x_col, y_col, x_label, y_label, title):
    """
    Tạo biểu đồ cột với dữ liệu tùy chỉnh cho trục x và trục y.

    Parameters:
    - dataframe (pd.DataFrame): Dữ liệu chứa thông tin cho biểu đồ.
    - x_col (str): Tên cột dữ liệu cho trục x.
    - y_col (str): Tên cột dữ liệu cho trục y.
    - x_label (str): Nhãn cho trục x.
    - y_label (str): Nhãn cho trục y.
    - title (str): Tiêu đề của biểu đồ.

    Returns:
    - dcc.Graph: Biểu đồ cột đã được cấu hình.
    """
    return dcc.Graph(
        id='bar-chart',
        figure=px.bar(dataframe, x=x_col, y=y_col, 
                      labels={x_col: x_label, y_col: y_label},
                      title=title)
            .update_layout(
                title=title,
                xaxis_title=x_label,
                yaxis_title=y_label,
                xaxis=dict(
                    tickformat='%d/%m/%y',
                    tickmode='linear',
                    tickangle=-45,
                    showgrid=True,
                    gridwidth=1,
                    #gridcolor='rgb(255, 203, 2)'  # Màu đường lưới
                ),
                yaxis=dict(
                    title=y_label,
                    rangemode='tozero',
                    showgrid=True,
                    gridwidth=1,
                    #gridcolor='rgb(255, 203, 2)'  # Màu đường lưới
                ),
                #plot_bgcolor='rgba(255, 203, 2, 0.7)',
                #paper_bgcolor='rgba(230, 230, 230, 0.5)'  # Màu nền giấy
            )
            .update_traces(
                marker=dict(
                    #color='rgb(0, 39, 76)',  # Màu cột
                    line=dict(
                        #color='rgb(0, 39, 76)',  # Màu viền cột
                        width=1.5
                    )
                ),
                texttemplate='%{y}',
                textposition='outside',
                cliponaxis=False,
                hovertemplate='Ngày: %{x}<br>%{y_label}: %{y}<extra></extra>'
            )
    )


def create_area_chart_new(df, x_col, y_col, x_label, y_label, chart_title):
    fig = px.area(df.reset_index(), x=x_col, y=y_col,
                  labels={x_col: x_label, y_col: y_label},
                  title=chart_title,
                  color_discrete_sequence=['#2B3467'],  # Đặt màu vùng
                 )
    # Cấu hình lại layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {
                'family': 'Arial',  # Font chữ cho tiêu đề
                'size': 25,  # Kích thước chữ cho tiêu đề (có thể điều chỉnh theo ý muốn)
                'color': '#89182c'  # Màu chữ cho tiêu đề
            },
            'x': 0.5,  # Căn giữa tiêu đề
            'xanchor': 'center',  # Căn giữa theo trục x
            'y': 0.95,
            'yanchor': 'top'  # Căn trên theo trục y
        },
        xaxis_title="",  # Ẩn tiêu đề trục x
        yaxis_title="",  # Ẩn tiêu đề trục y
        plot_bgcolor='#fffbea',  # Màu nền của biểu đồ
        margin=dict(l=10, r=30, t=50, b=10),
        height=400, 
    )
    return fig


def create_bar_chart_new(df, x_col, y_col, x_label, y_label, chart_title):
    fig = px.bar(df.reset_index(), x=x_col, y=y_col,
                 labels={x_col: x_label, y_col: y_label},
                 title=chart_title,
                 color_discrete_sequence=['#2B3467'],  # Đặt màu cột
                 text_auto=True
                )

    # Cấu hình lại layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {
                'family': 'Arial',  # Font chữ cho tiêu đề
                'size': 25,  # Kích thước chữ cho tiêu đề (có thể điều chỉnh theo ý muốn)
                'color': '#89182c'  # Màu chữ cho tiêu đề
            },
            'x': 0.5,  # Căn giữa tiêu đề
            'xanchor': 'center',  # Căn giữa theo trục x
            'y': 0.95,
            'yanchor': 'top'  # Căn trên theo trục y
        },
        xaxis_title="",  # Ẩn tiêu đề trục x
        yaxis_title="",  # Ẩn tiêu đề trục y
        plot_bgcolor='#fffbea',  # Màu nền của biểu đồ
        margin=dict(l=10, r=30, t=50, b=10),
        height=400, 
    )
    
    fig.update_traces(
        textposition='outside',  # Hoặc 'inside' nếu muốn bên trong
        textfont=dict(color='#2B3467'),  # Đặt màu chữ cho tất cả các nhãn
        textangle=0  # Đặt góc hiển thị nhãn là 90 độ (chiều dọc)
    )
    return fig


def MetricCard_new(title, value):
    return dbc.Card(
        dbc.CardBody(
            [
                html.P(value, className='card-text', style={'fontFamily': 'Arial', 'textAlign': 'center', 'fontSize': '30px', 'fontWeight': 'bold'}),
                html.H4(title, className='card-title', style={'fontFamily': 'Arial', 'textAlign': 'center', 'fontSize': '25px', 'color': '#89182c'}),
            ]
        ),
        style={
            'height': '150px',
            'marginBottom': '20px',
            'borderRadius': '10px',
            'border': '3px solid #2B3467',  # Border styling
            'backgroundColor': '#fffbea',  # Background color
            'padding': '5px',  # Padding inside card
        }
    )

[('anhln', 'anhln', datetime.datetime(2024, 6, 20, 21, 32, 12, 514436), datetime.datetime(2024, 6, 20, 21, 32, 12, 514436), 'app-hmlXEGA8Ht4GG829rQOvwJqT'), ('ducbq', 'ducbq', datetime.datetime(2024, 6, 14, 0, 11, 25, 85305), datetime.datetime(2024, 6, 14, 0, 11, 25, 85305), 'app-CNFjTcU191SEKvpdyvVwgKeO'), ('ducna', 'ducna', datetime.datetime(2024, 6, 20, 21, 32, 12, 514436), datetime.datetime(2024, 6, 20, 21, 32, 12, 514436), 'app-hmlXEGA8Ht4GG829rQOvwJqT'), ('phuongpd', 'phuongpd', datetime.datetime(2024, 6, 4, 21, 53, 37, 602236), datetime.datetime(2024, 6, 4, 21, 53, 37, 602236), 'app-CNFjTcU191SEKvpdyvVwgKeO'), ('sonnt', 'Nguyen Trung Son', datetime.datetime(2024, 6, 20, 22, 43, 41, 727599), datetime.datetime(2024, 6, 20, 22, 43, 41, 727599), 'app-hmlXEGA8Ht4GG829rQOvwJqT'), ('tungtk', 'tungtk', datetime.datetime(2024, 6, 20, 21, 32, 12, 514436), datetime.datetime(2024, 6, 20, 21, 32, 12, 514436), 'app-CNFjTcU191SEKvpdyvVwgKeO'), ('vyvt', 'vyvt', datetime.datetime(2024, 6, 20, 21

In [642]:
conn = connect_db()

transcripts_df = fetch_data_from_table(conn, "transcripts")
feedback_df = fetch_data_from_table(conn, "feedback")
users_df = fetch_data_from_table(conn, "users")
bot_information_df = fetch_data_from_table(conn, "bot_information")
sessions_df = fetch_data_from_table(conn, "sessions")
conversations_df = fetch_data_from_table(conn, "conversations")
conversation_logs_df = fetch_data_from_table(conn, "conversation_logs")
error_logs_df = fetch_data_from_table(conn, "error_logs")

#Xử lý của Minh

In [643]:
# Part 1: conversation_logs_df
conversation_logs_df['timestamp'] = pd.to_datetime(conversation_logs_df['timestamp'])
conversation_logs_df['created_at'] = pd.to_datetime(conversation_logs_df['created_at'])
# Ensure 'response_speed' is in seconds
conversation_logs_df['response_speed'] = (conversation_logs_df['timestamp'] - conversation_logs_df['created_at']).dt.total_seconds()

# Part 2: Error_logs_df
error_conditions = ['sorry', 'no information', 'fail', 'wrong', 'error', 'not found', 'unable to', 'cannot', 'unsuccessful']
error_logs_2_df = conversation_logs_df[conversation_logs_df['outputs'].str.contains('|'.join(error_conditions), case=False)]

# Creating the Error_logs_df
error_logs_2_df = error_logs_2_df[['message_id', 'session_id', 'inputs', 'outputs', 'timestamp', 'user_id', 'conversation_id', 'created_at', 'updated_at']].copy()
error_logs_2_df.columns = ['error_id', 'session_id', 'input_message', 'error_message', 'timestamp', 'user_id', 'conversation_id', 'created_at', 'updated_at']
error_logs_2_df

# Part 3: session_df

# Convert timestamps to datetime objects
conversation_logs_df['timestamp'] = pd.to_datetime(conversation_logs_df['timestamp'])
conversation_logs_df['created_at'] = pd.to_datetime(conversation_logs_df['created_at'])
errored_sessions = error_logs_df['session_id'].unique()
conversation_logs_df['Error_status'] = conversation_logs_df['session_id'].apply(lambda x: 1 if x in errored_sessions else 0)

# Calculate session start and end times based on the first and last message timestamps
session_start_times = conversation_logs_df.groupby('session_id')['timestamp'].min().reset_index()
session_start_times.columns = ['session_id', 'start_time']
session_end_times = conversation_logs_df.groupby('session_id')['timestamp'].max().reset_index()
session_end_times.columns = ['session_id', 'end_time']

# Merge start and end times into a single DataFrame
session_times = pd.merge(session_start_times, session_end_times, on='session_id')

# Calculate session durations
session_times['session_duration'] = (session_times['end_time'] - session_times['start_time']).dt.total_seconds()

# Merge with original sessions details to get user_id, created_at, and updated_at
sessions = pd.read_csv('sessions.csv')  # Read the original sessions file
session_extended_df = pd.merge(session_times, sessions[['session_id', 'user_id', 'created_at', 'updated_at']], on='session_id', how='left')

# Calculate error rates for each session
error_counts = error_logs_2_df.groupby('session_id').size().reset_index(name='error_count')
message_counts = conversation_logs_df.groupby('session_id').size().reset_index(name='message_count')

# Merge error counts and message counts
error_rate_df = pd.merge(error_counts, message_counts, on='session_id', how='right')
error_rate_df['error_rate'] = (error_rate_df['error_count'] / error_rate_df['message_count']).fillna(0) * 100

# Merge error rate into session_extended_df
session_extended_df = pd.merge(session_extended_df, error_rate_df[['session_id', 'error_rate']], on='session_id', how='left')


session_df = session_extended_df[['session_id', 'user_id', 'start_time', 'end_time', 'session_duration', 'created_at', 'updated_at', 'error_rate']].copy()

# Part 5: Non_FAQs_df
non_faqs_cond = (conversation_logs_df['outputs'].str.contains('sorry', case=False) | 
                conversation_logs_df['outputs'].str.contains('no information', case=False) | 
                conversation_logs_df['domain'].isna())

non_faqs_df = conversation_logs_df[non_faqs_cond]

# Add error messages to non_faqs
non_faqs_df = pd.concat([non_faqs_df, error_logs_df.rename(columns={'error_id': 'message_id', 'input_message': 'inputs', 'error_message': 'outputs'})])

# Creating the Non_FAQs_df
non_faqs_df = non_faqs_df[['message_id', 'inputs', 'outputs', 'domain']].copy()
non_faqs_df['usage_count'] = non_faqs_df.groupby('inputs')['inputs'].transform('count')
non_faqs_df = non_faqs_df[['message_id', 'inputs', 'domain', 'usage_count', 'outputs']]
non_faqs_df.columns = ['question_id', 'text_content', 'domain', 'usage_count', 'outputs']
non_faqs_df_used = non_faqs_df[['text_content', 'usage_count', 'outputs']]

# Sort by 'usage_count' in descending order
non_faqs_df_used = non_faqs_df_used.sort_values(by='usage_count', ascending=False)
non_faqs_df_used = non_faqs_df_used.drop_duplicates(subset='text_content', keep='first')


# Part 6: Peak Time
# Convert timestamp to datetime
conversation_logs_df['timestamp'] = pd.to_datetime(conversation_logs_df['timestamp'])

# Extract hour and day from timestamp
conversation_logs_df['hour'] = conversation_logs_df['timestamp'].dt.hour
conversation_logs_df['date'] = conversation_logs_df['timestamp'].dt.strftime('%d-%m-%Y')
feedback_df['date'] = feedback_df['created_at'].dt.strftime('%d-%m-%Y')

# Count the number of messages received in each hour and day
peak_time = conversation_logs_df.groupby(['date', 'hour']).size().reset_index(name='message_count')
conversation_logs_df['date'] = conversation_logs_df['date'].astype(str)

# Use hour as a number from 0 to 23
peak_time['hour'] = peak_time['hour'].apply(lambda x: x)


#feedback
feedback_subset = feedback_df[['session_id', 'feedback_type', 'feedback_text']]
conversation_logs_df = pd.merge(conversation_logs_df, feedback_subset, on='session_id', how='left')
# Tính toán số lượng mỗi loại feedback_type
# Thay thế NaN bằng "No Feedback"
conversation_logs_df['feedback_type'].fillna('No Feedback', inplace=True)
feedback_counts = conversation_logs_df['feedback_type'].value_counts()

# Chuyển đổi thành DataFrame để dễ dàng sử dụng với plotly
feedback_df_1 = feedback_counts.reset_index()
feedback_df_1.columns = ['feedback_type', 'count']


In [644]:
# Calculate the total number of messages
total_messages = len(conversation_logs_df)

# Calculate the number of error messages
error_messages = conversation_logs_df['Error_status'].sum()

# Calculate the percentage of error messages
error_rate = (error_messages / total_messages) * 100
non_error_rate = 100 - error_rate

# Prepare data for the pie chart
labels = ['Error Messages', 'Non-Error Messages']
values = [error_rate, non_error_rate]

# Create the pie chart using Plotly Express
pie_chart = px.pie(
    names=labels, 
    values=values, 
    title='Error Rate',
    color_discrete_sequence=['#ff9999', '#66b3ff']
)

In [649]:
conversation_logs_df.to_excel('124.xlsx')

/var/folders/nw/l02js6ln7y91b_vfz77rnjtm0000gn/T/ipykernel_26119/1749924445.py:226: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/var/folders/nw/l02js6ln7y91b_vfz77rnjtm0000gn/T/ipykernel_26119/1749924445.py:227: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/var/folders/nw/l02js6ln7y91b_vfz77rnjtm0000gn/T/ipykernel_26119/1749924445.py:234: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/var/fo

In [647]:
# Data processing for metrics and charts (simplified)

# Entired data showed in card
total_sessions = conversation_logs_df['session_id'].nunique()
total_users = conversation_logs_df['user_id'].nunique()
total_messages = conversation_logs_df['message_id'].count()
avg_response_time = round(conversation_logs_df['response_speed'].mean(),2)
avg_session_duration = round(session_df['session_duration'].mean(),2)
avg_error_rate = round(session_df['error_rate'].mean(),2)


# Aggregate data by date or hour
conversation_logs_df['date'] = pd.to_datetime(conversation_logs_df['timestamp']).dt.date
conversation_logs_df['hour'] = pd.to_datetime(conversation_logs_df['timestamp']).dt.hour
session_df['date'] = pd.to_datetime(session_df['start_time']).dt.date


message_by_user = conversation_logs_df.groupby('date')['message_id'].nunique()
users_by_date = conversation_logs_df.groupby('date')['user_id'].nunique()
messages_by_hour = conversation_logs_df.groupby('hour')['message_id'].count()
#messages_by_hour_mean = messages_by_hour.mean()

# Create charts
fig1 = create_area_chart_new(message_by_user, 'date', 'message_id', 'Date', 'Message Count', 'Số lượng tin nhắn của người dùng')
fig2 = create_bar_chart_new(users_by_date, 'date', 'user_id', 'Date', 'Unique Users', 'Số lượng người dùng truy cập chatbot')
fig3 = create_bar_chart_new(messages_by_hour, 'hour', 'message_id', 'Hour of Day', 'Message Count', 'Số lượng tin nhắn theo khung giờ')
# Tạo biểu đồ hình tròn
# Customize colors and optimize layout for pie chart
# Customize colors and optimize layout for pie chart
fig4 = px.pie(
    feedback_df_1, 
    values='count', 
    names='feedback_type', 
    title='Tỷ lệ phản hồi người dùng',
    color = feedback_df_1.index,
    color_discrete_map={
        1: '#9E9EAC',  #like
        2: '#89182c', #dislike
        0: '#2B3467' #no feedback
    }
)

# Update layout for title, legend, and size
fig4.update_layout(
    title={
        'text': 'Tỷ lệ phản hồi người dùng',
        'x': 0.5,  # Center title
        'xanchor': 'center',  # Align the title to the center
        'font': {
            'family': 'Arial',  # Font family
            'size': 25,  # Title size
            'color': '#89182c',  # Title color
        }
    },
    legend={
        'orientation': 'h',  # Horizontal legend
        'yanchor': 'bottom',  # Align to bottom
        'y': -0.5,  # Position below the chart
        'xanchor': 'center',
        'x': 0.5
    },
    margin=dict(
        l=5,  # Left padding
        r=5,  # Right padding
        t=70,  # Top padding
        b=50 
    ),  # Adjust top margin to fit title
    paper_bgcolor='#fffbea',
    height=400,  # Increase pie chart height
    width=350,  # Increase pie chart width
)

# Ensure it's a pie chart (not a donut)
fig4.update_traces(
    textinfo='percent',  # Display percentage and label
    textfont_size=18  # Adjust font size for labels
)

card1 = MetricCard_new("Tổng số phiên chat", total_sessions),
card2 = MetricCard_new("Tổng số tin nhắn", total_messages),
card3 = MetricCard_new("Thời lượng chat trung bình", avg_session_duration),
card4 = MetricCard_new("Tỷ lệ lỗi chatbot", avg_error_rate),
card5 = MetricCard_new("Tốc độ phản hồi", avg_response_time)

In [648]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    html.Div([
        html.H1(
            "PV POWER - TRUNG TÂM QUẢN TRỊ CHATBOT", 
            style={
                'textAlign': 'center',
                'background': 'linear-gradient(90deg, rgba(37,64,109,1) 0%, rgba(137,24,44,1) 100%)',
                'width': '1450px',
                'height': '100px',
                'flexShrink': '0',
                'display': 'flex',
                'alignItems': 'center',
                'justifyContent': 'center',
                'color': '#FFF',
                'fontFamily': 'Arial',
                'fontSize': '30px',
                'fontStyle': 'normal',
                'fontWeight': '700',
                'lineHeight': 'normal',
                'marginBottom': '20px'
            }
        ),
        html.H2(
            "Hoạt động và Lưu lượng của Chatbot", 
            style={
                'textAlign': 'left',
                #'background': 'linear-gradient(90deg, #71D0EC 0%, #1482BF 100%)',
                'width': '1450px',
                'height': '100px',
                'flexShrink': '0',
                'display': 'flex',
                'alignItems': 'center',
                'justifyContent': 'left',
                'color': '#2B3467',
                'fontFamily': 'Arial',
                'fontSize': '30px',
                'fontStyle': 'normal',
                'fontWeight': '700',
                'lineHeight': 'normal',
                'marginBottom': '5px'
            }
        ),
        #Bộ lọc
        html.Div([
            # Dropdown cho chọn người dùng
            dcc.Dropdown(
                id='dropdown-user',
                options=[{'label': user, 'value': user} for user in conversation_logs_df['user_id'].unique()],
                value=None,  # Giá trị mặc định (không lọc nếu None)
                placeholder="Chọn người dùng",
                style={
                    'width': '300px', 
                    'height': '50px', 
                    'marginBottom': '10px', 
                    'borderRadius': '10px',  # Viền bo góc
                    #'border': '3px solid #2B3467',  # Viền màu
                    #'backgroundColor': '#fffbea'  # Màu nền
                }
            ),
            # Date picker cho chọn khoảng thời gian
            dcc.DatePickerRange(
                id='date-picker-range',
                start_date=conversation_logs_df['date'].min(),
                end_date=conversation_logs_df['date'].max(),
                display_format='DD-MM-YYYY',
                style={
                    'width': '300px', 
                    'height': '50px', 
                    'marginBottom': '10px', 
                    'borderRadius': '10px',  # Viền bo góc
                    #'border': '2px solid #2B3467',  # Viền màu
                    #'backgroundColor': '#fffbea'  # Màu nền
                }
            )
        ], style={
            'display': 'flex', 
            'width': '1450px',
            'justifyContent': 'right', 
            'alignItems': 'center', 
            'marginBottom': '10px'
        }),

        # Flex layout with two columns
        html.Div(
            style={'display': 'flex', 'width': '1450px', 'column_gap': '10%', 'alignItems': 'stretch', 'margin': '0'},
            children=[
                # Left column (smaller, containing metric cards with data)
                html.Div(
                    style={'flex': '1', 'padding': '10px', 'marginTop': '10px'},
                    children=[
                        html.Div(id='total-sessions-card', children=card1),
                        html.Div(id='total-messages-card', children=card2),
                        html.Div(id='avg-session-duration-card', children=card3),
                        html.Div(id='error-rate-card', children=card4),
                        html.Div(id='avg-response-time-card', children=card5),
                    ]
                ),
                
                # Right column (larger, containing bar charts)
                html.Div(
                    style={'flex': '3', 'padding': '20px', 'marginTop': '1px'},
                    children=[
                        dcc.Graph(
                            id='bar-chart-messages-users',
                            figure= fig1,
                            style={'padding': '0px','border': '3px solid #2B3467', 'borderRadius': '10px', 'marginBottom': '20px', 'overflow': 'hidden'}
                        ),
                        dcc.Graph(
                            id='bar-chart-messages-time',
                            figure= fig3,
                            style={'padding': '0px','border': '3px solid #2B3467', 'borderRadius': '10px', 'marginBottom': '10px', 'overflow': 'hidden'}
                        )
                    ]
                )
            ]
        ),
        html.H2(
            "Hoạt động của người dùng", 
            style={
                'textAlign': 'left',
                #'background': 'linear-gradient(90deg, #71D0EC 0%, #1482BF 100%)',
                'width': '1450px',
                'height': '100px',
                'flexShrink': '0',
                'display': 'flex',
                'alignItems': 'center',
                'justifyContent': 'left',
                'color': '#2B3467',
                'fontFamily': 'Arial',
                'fontSize': '30px',
                'fontStyle': 'normal',
                'fontWeight': '700',
                'lineHeight': 'normal',
                'marginBottom': '20px'
            }
        ),
        
        html.Div(
            style={'display': 'flex', 'width': '1450px', 'column_gap': '10%', 'alignItems': 'stretch', 'margin': '0'},
            children=[
                # Left column (smaller, containing metric cards with data)
                html.Div(
                    style={'flex': '1', 'padding': '10px', 'marginTop': '20px'},
                    children=[
                        dcc.Graph(
                            id='pie-chart-feedback',
                            figure= fig4,
                            style={'padding': '0px','border': '3px solid #2B3467', 'borderRadius': '10px', 'marginBottom': '0', 'overflow': 'hidden'}
                        )
                    ]
                ),
                
                # Right column (larger, containing bar charts)
                html.Div(
                    style={'flex': '3', 'padding': '20px', 'marginTop': '10px'},
                    children=[
                        dcc.Graph(
                            id='bar-chart-session-users',
                            figure= fig2,
                            style={'padding': '0px','border': '3px solid #2B3467', 'borderRadius': '10px', 'marginBottom': '0', 'overflow': 'hidden'}
                        )
                    ]
                )
            ]
        ),
        html.Div([
                    html.H4("Những câu hỏi thường gặp", 
                    style={
                        'textAlign': 'left',
                        #'background': 'linear-gradient(90deg, #71D0EC 0%, #1482BF 100%)',
                        'width': '1450px',
                        'height': '100px',
                        'flexShrink': '0',
                        'display': 'flex',
                        'alignItems': 'center',
                        'justifyContent': 'left',
                        'color': '#2B3467',
                        'fontFamily': 'Arial',
                        'fontSize': '30px',
                        'fontStyle': 'normal',
                        'fontWeight': '700',
                        'lineHeight': 'normal',
                        'marginBottom': '20px'
                    }
                    ),
                    # List FAQs here
                    dash_table.DataTable(
                    columns=[{"name": "Câu hỏi", "id": "text_content"},
                            {"name": "Số lượt hỏi", "id": "usage_count"}
                    ],    
                    #columns=[{"name": i, "id": i} for i in non_faqs_df_used.columns],
                    data=non_faqs_df_used.to_dict('records'),
                    page_size=20,  # Number of rows per page
                    style_table={'overflowX': 'auto', 'width': '1450px'},  # To handle overflow
                    style_cell={'textAlign': 'left','border': '1px solid #2B3467','whiteSpace': 'normal','height': 'auto', 'fontFamily': 'Arial' },  # Align text to the left
                    style_header={'fontWeight': 'bold','height': '40px', 'textAlign': 'center','backgroundColor': '#2B3467','color': 'white','border': '1px solid #2B3467', 'fontSize': '20px'} # Bold headers
                    )
                ], 
            className='faq-section'
        )
    ])
])

@app.callback(
    [
        Output('total-sessions-card', 'children'),
        Output('total-messages-card', 'children'),
        Output('avg-session-duration-card', 'children'),
        Output('error-rate-card', 'children'),
        Output('avg-response-time-card', 'children'),
        Output('bar-chart-messages-users', 'figure'),
        Output('bar-chart-messages-time', 'figure'),
        Output('bar-chart-session-users', 'figure'),
        Output('pie-chart-feedback', 'figure')
    ],
    [
        Input('dropdown-user', 'value'),
        Input('date-picker-range', 'start_date'),
        Input('date-picker-range', 'end_date')
    ]
)
def update_metrics_and_charts(selected_user, start_date, end_date):
    # Filter conversation_logs_df based on user and date range
    filtered_conversation_logs = conversation_logs_df[
        (conversation_logs_df['date'] >= pd.to_datetime(start_date)) &
        (conversation_logs_df['date'] <= pd.to_datetime(end_date))
    ]
    if selected_user:
        filtered_conversation_logs = filtered_conversation_logs[filtered_conversation_logs['user_id'] == selected_user]

    # Filter session_df similarly
    filtered_sessions_df = session_df[
        (session_df['date'] >= pd.to_datetime(start_date)) &
        (session_df['date'] <= pd.to_datetime(end_date))
    ]
    if selected_user:
        filtered_sessions_df = filtered_sessions_df[filtered_sessions_df['user_id'] == selected_user]

    # Update metric cards with filtered data
    total_sessions_filtered = filtered_conversation_logs['session_id'].nunique()
    total_messages_filtered = filtered_conversation_logs['message_id'].count()
    avg_session_duration_filtered = round(filtered_sessions_df['session_duration'].mean(), 2)
    avg_error_rate_filtered = round(filtered_sessions_df['error_rate'].mean(), 2)
    avg_response_time_filtered = round(filtered_conversation_logs['response_speed'].mean(), 2)

    # Create updated charts
    card1 = MetricCard_new("Tổng số phiên chat", total_sessions_filtered),
    card2 = MetricCard_new("Tổng số tin nhắn", total_messages_filtered),
    card3 = MetricCard_new("Thời lượng chat trung bình", avg_session_duration_filtered),
    card4 = MetricCard_new("Tỷ lệ lỗi chatbot", avg_error_rate_filtered),
    card5 = MetricCard_new("Tốc độ phản hồi", avg_response_time_filtered)

    # Aggregate data for charts
    message_by_user = filtered_conversation_logs.groupby('date')['message_id'].nunique()
    users_by_date = filtered_conversation_logs.groupby('date')['user_id'].nunique()
    messages_by_hour = filtered_conversation_logs.groupby('hour')['message_id'].count()

    # Create updated charts
    fig1 = create_area_chart_new(message_by_user, 'date', 'message_id', 'Date', 'Message Count', 'Số lượng tin nhắn của người dùng')
    fig2 = create_bar_chart_new(users_by_date, 'date', 'user_id', 'Date', 'Unique Users', 'Số lượng người dùng truy cập chatbot')
    fig3 = create_bar_chart_new(messages_by_hour, 'hour', 'message_id', 'Hour of Day', 'Message Count', 'Số lượng tin nhắn theo khung giờ')

    # Update pie chart (feedback)
    feedback_counts = filtered_conversation_logs['feedback_type'].value_counts()

    # Chuyển đổi thành DataFrame để dễ dàng sử dụng với plotly
    feedback_df_2 = feedback_counts.reset_index()
    feedback_df_2.columns = ['feedback_type', 'count']
    fig4 = px.pie(
        feedback_df_2, 
        values='count', 
        names='feedback_type', 
        title='Tỷ lệ phản hồi người dùng',
        color = feedback_df_2.index,
        color_discrete_map={
            1: '#9E9EAC',  #like
            2: '#89182c', #dislike
            0: '#2B3467' #no feedback
        }
    )
    fig4.update_layout(
        title={
            'text': 'Tỷ lệ phản hồi người dùng',
            'x': 0.5,  
            'xanchor': 'center',
            'font': {'family': 'Arial', 'size': 25, 'color': '#89182c'}
        },
        legend={
            'orientation': 'h', 
            'yanchor': 'bottom', 
            'y': -0.5,  
            'xanchor': 'center', 
            'x': 0.5
        },
        margin=dict(l=5, r=5, t=70, b=50),
        paper_bgcolor='#fffbea',
        height=400,
        width=350
    )
    fig4.update_traces(textinfo='percent', textfont_size=18)

    # Return updated metric values and figures
    return (
        card1,
        card2,
        card3,
        card4,
        card5,
        fig1,
        fig3,
        fig2,
        fig4
    )

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


/var/folders/nw/l02js6ln7y91b_vfz77rnjtm0000gn/T/ipykernel_26119/1749924445.py:226: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/var/folders/nw/l02js6ln7y91b_vfz77rnjtm0000gn/T/ipykernel_26119/1749924445.py:227: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/var/folders/nw/l02js6ln7y91b_vfz77rnjtm0000gn/T/ipykernel_26119/1749924445.py:234: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/var/fo